In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from collections import deque
import pandas as pd
import math


import sys
sys.path.append('..')
# from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

True

In [2]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True],
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FALCO'],
    'character_name': ['PIKACHU', 'PICHU'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

In [3]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())
dataset.dataset.head()

PIKACHU    4096
PICHU       230
Name: labels, dtype: int64


/workspace/slp_jaspar/pichu_pikachu_classification/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


,stage_name,num_players,conclusive,player_character_name,player_type_name,opposing_player_type_name,player_inputs_np_sub_path,length,labels
0,FINAL_DESTINATION,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\daa0e300-d731-42ea-a634-28c9db71...,7244,PIKACHU
1,POKEMON_STADIUM,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\142d032c-4d4a-4056-b9a9-aca37c88...,7384,PIKACHU
2,YOSHIS_STORY,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\3a508695-3da7-464a-b228-88d0f051...,14390,PIKACHU
3,FOUNTAIN_OF_DREAMS,2,True,PIKACHU,HUMAN,HUMAN,mango\PIKACHU\1be3107a-58a6-4f48-80c0-191cf926...,11363,PIKACHU
4,BATTLEFIELD,2,True,PICHU,HUMAN,HUMAN,mango\PICHU\36e5d441-a59c-411f-a381-c233d8b5e3...,7817,PICHU


In [4]:
labels_order =  dataset.number_of_segments_per_game(3600,1000)
print(labels_order)
labels_order = labels_order['Label'].values

     Label  Count  Shift
0  PIKACHU   4067  30486
1    PICHU    227   1648


In [5]:
X_train, X_test, y_train, y_test  = dataset.train_test_split_numpy(test_ratio = .20, val = False)
print()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)



100%|██████████| 854/854 [00:00<00:00, 6167.37it/s]



(1600, 9, 3600)
(1600,)
(400, 9, 3600)
(400,)


In [6]:
le = LabelEncoder()
print(y_train)
le.fit(['PIKACHU','PICHU'])
Y_train_encode = le.transform(y_train)
Y_test_encode = le.transform(y_test)
# print(Y_train_encode)

['PIKACHU' 'PIKACHU' 'PIKACHU' ... 'PICHU' 'PICHU' 'PICHU']


In [8]:
X_train_tensor = torch.tensor(X_train)
X_test_tensor = torch.tensor(X_test)
y_train_tensor = torch.tensor(Y_train_encode)
y_test_tensor = torch.tensor(Y_test_encode)

In [9]:
from hydra.code.hydra_multivariate import HydraMultivariate
from hydra.code.hydra import SparseScaler
from sklearn.linear_model import RidgeClassifierCV

In [10]:
model = HydraMultivariate(3600,9)

In [11]:
X_training_transform = SparseScaler.transform(X_train_tensor)
X_test_transform = SparseScaler.transform(X_test_tensor)
scaler = SparseScaler()

TypeError: SparseScaler.transform() missing 1 required positional argument: 'X'